In [217]:
# https://western-sky.tistory.com/49

In [227]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [228]:
service = Service(executable_path=ChromeDriverManager().install())

# 한 페이지, 여러페이지 베스트 댓글 추출(셀레니움)

In [229]:
# 데이터 프레임 생성
df = pd.DataFrame(data=[], columns=['회차','날짜','댓글','답글','좋아요','싫어요'])

In [232]:
# 하나의 에피소드 수집
def collector(service, df, webtoon_id, episode_no):
    TIMEOUT = 1
    #driver = webdriver.Chrome("C:\\Temp\\chromedriver") # 환경에 맞게 수정
    driver = webdriver.Chrome(service=service)
    
    url = 'https://comic.naver.com/webtoon/detail?'
    episode_url = url + 'titleId=' + str(webtoon_id) + '&no='+str(episode_no)
    print(episode_url)
    driver.get(episode_url) # 새창 열기
    driver.implicitly_wait(TIMEOUT)
    
    # 댓글창에 해당하는 프레임 선택
    driver.switch_to.frame('commentIframe')

    dates = driver.find_elements(By.CSS_SELECTOR, '.u_cbox_date')
    reviews = driver.find_elements(By.CSS_SELECTOR, '.u_cbox_text_wrap')
    points = driver.find_elements(By.CSS_SELECTOR, '.u_cbox_tool')

    for i in range(len(reviews)):
        #  ['episode 회차','날짜','댓글','답글','좋아요','싫어요']
        temp = []
        temp.append(episode_no)
        temp.append(dates[i].text)
        temp.append(reviews[i].text.replace('BEST','')) # BEST 제거

        # normal case   : 답글\n66\n좋아요\n62661\n싫어요\n542  
        # abnormal case : 답글\n좋아요\n62661\n싫어요\n542
        points_text = points[i].text.split('\n')
        points_len = len(points_text)
        
        if points_len >= 2 and points_text[1].isdigit():
            temp.append(points_text[1])
        else:
            temp.append(0)
            
        if points_len >= 4 and points_text[3].isdigit():
            temp.append(points_text[3])
        else:
            temp.append(0)
            
        if points_len >= 6 and points_text[5].isdigit():
            temp.append(points_text[5])
        else:
            temp.append(0)        

        temp = pd.DataFrame(data=[temp], columns=df.columns)
        df = pd.concat([df,temp])
    #df.reset_index(drop=True)
    return df

In [233]:
# 1페이지 테스트
df = collector(service, df, 747269, 1)
df

https://comic.naver.com/webtoon/detail?titleId=747269&no=1


회차                날짜                                                 댓글  답글  \
0  1  2020-05-26 23:01                                          나 닉값해도 되냐  66   
0  1  2020-05-26 23:02  먼저 소설 보신 분들~ 다들 굉장히 감격스럽고 기대되는 장면이 있으시겠죠? 저도 그...  29   
0  1  2020-05-26 23:01  「오직 나만이, 이 세계의 결말을 알고있다.」 레전드 갓작의 등장입니다.. 여러분 ...  29   
0  1  2020-05-26 23:07   이 소설 멸살법 표절 아닌가요? 하 저번에도 s뭐시기 표절 있던데 표절좀 그만하세요;;  76   
0  1  2020-05-26 23:14              표절 얘기 드립이에요 ㅎㅎ 작중 나오는 소설 멸살법 표절 드립입니다  12   
0  1  2020-05-26 23:01         전독시 웹툰 진짜 퀄리티 미쳤다 너무 잘나왔다... 눈물이 앞을 가리네요..   8   
0  1  2020-05-26 23:15  소설 안본 놈들이 벨이라고 까더라ㅋㅋㅋㅋ이건 김컴 주인공들이 서로가 서로를 구원하는...  20   
0  1  2020-05-26 23:10                                    다들 제 닉처럼 행동해주세요  19   
0  1  2020-05-26 23:13  눈 크게뜨고 봐라.고개 돌리지 말고 봐라. 입 다물고 봐라. 이 소설 이상한 애들이...  13   
0  1  2020-05-26 23:12                       비엘 드립 치지마세요 님들만 웹툰 보는거 아님...   8   
0  1  2020-05-26 23:50  나 오늘 이작품 첨 보는데 소설로 엄청 유명한가봐요? 전 여자인데 여자들이 봐도 재...  34   
0  1  2020-05-26 23:16     드디어ㅠㅠ 표절 드립이예요! 소설 원작입니다ㅠㅠ 진짜 재밌게 잘 읽은 작품이예요ㅠㅠ   3   
0  1  2020-05-26 23:06                                여러분 이거 개띵작이에요 꼭 보세요   8   
0  1  2020-05-29 21:04  이 작품 시리즈와 문피아에서 원작 구매 성비 확인해보면 둘 다 남성 비율이 더 높은...  21   
0  1  2021-08-03 20:46  #8월의기적_유중혁_생일축하해#HappyJoongHyeokDay_2021 #Happ...  32   
0  1  2020-05-26 23:01                                          나 닉값해도 되냐  66   
0  1  2020-05-26 23:02  먼저 소설 보신 분들~ 다들 굉장히 감격스럽고 기대되는 장면이 있으시겠죠? 저도 그...  29   
0  1  2020-05-26 23:01  「오직 나만이, 이 세계의 결말을 알고있다.」 레전드 갓작의 등장입니다.. 여러분 ...  29   
0  1  2020-05-26 23:07   이 소설 멸살법 표절 아닌가요? 하 저번에도 s뭐시기 표절 있던데 표절좀 그만하세요;;  76   
0  1  2020-05-26 23:14              표절 얘기 드립이에요 ㅎㅎ 작중 나오는 소설 멸살법 표절 드립입니다  12   
0  1  2020-05-26 23:01         전독시 웹툰 진짜 퀄리티 미쳤다 너무 잘나왔다... 눈물이 앞을 가리네요..   8   
0  1  2020-05-26 23:15  소설 안본 놈들이 벨이라고 까더라ㅋㅋㅋㅋ이건 김컴 주인공들이 서로가 서로를 구원하는...  20   
0  1  2020-05-26 23:10                                    다들 제 닉처럼 행동해주세요  19   
0  1  2020-05-26 23:13  눈 크게뜨고 봐라.고개 돌리지 말고 봐라. 입 다물고 봐라. 이 소설 이상한 애들이...  13   
0  1  2020-05-26 23:12                       비엘 드립 치지마세요 님들만 웹툰 보는거 아님...   8   
0  1  2020-05-26 23:50  나 오늘 이작품 첨 보는데 소설로 엄청 유명한가봐요? 전 여자인데 여자들이 봐도 재...  34   
0  1  2020-05-26 23:16     드디어ㅠㅠ 표절 드립이예요! 소설 원작입니다ㅠㅠ 진짜 재밌게 잘 읽은 작품이예요ㅠㅠ   3   
0  1  2020-05-26 23:06                                여러분 이거 개띵작이에요 꼭 보세요   8   
0  1  2020-05-29 21:04  이 작품 시리즈와 문피아에서 원작 구매 성비 확인해보면 둘 다 남성 비율이 더 높은...  21   
0  1  2021-08-03 20:46  #8월의기적_유중혁_생일축하해#HappyJoongHyeokDay_2021 #Happ...  32   

     좋아요   싫어요  
0  62660   542  
0  47269   643  
0  34064   508  
0  30367  2289  
0  26485   373  
0  14252   243  
0  10757   900  
0  10619   244  
0  10306  1678  
0   8571   870  
0   7197   567  
0   6905   262  
0   6755   222  
0   4535   372  
0    729    98  
0  62660   542  
0  47269   643  
0  34064   508  
0  30367  2289  
0  26485   373  
0  14252   243  
0  10757   900  
0  10619   244  
0  10306  1678  
0   8571   870  
0   7197   567  
0   6905   262  
0   6755   222  
0   4535   372  
0    729    98

In [ ]:
# 전체 페이지 테스트 : 차이 no 만큼 open됨
# for page in range(1, 121):
#     df = collector(driver, df, 747269, page)    
# df.reset_index(drop=True)